# Predicting Home Prices in Ames, Iowa


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder
import plotly.express as px
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import f_oneway, kruskal
import math
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer
from scipy import stats
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline

In [2]:
df= pd.read_csv("train.csv")

# Part 1: Examining the Data


We are examining the data to see if there are any issues or any feature engineering we might need to do. 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
#df.fillna(0,inplace=True)

In [5]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


We define a new variable called PricePerSF. In general, the bigger the house the more expensive it is because naturally a bigger house requires more materials. What might be easier and perhaps better to calculate is the price per square foot of the house instead. I plan to run the regression for both variables and see which one is the better predictor. 

df['PricePerSF']=df['SalePrice']/df['GrLivArea']

In [6]:
def catGraph(cat):
    print(df[cat].value_counts())
    fig=px.box(df, x=cat, y='SalePrice', title = 'Price By '+cat, points="all")
    fig.update_yaxes( 
        title_text = "Sale Price",
        title_standoff = 10)
    fig.update_xaxes(
        title_text = cat,
        title_standoff = 10)
    fig.show()    
    
    fig=px.box(df, x=cat, y='PricePerSF', title = 'Price By '+cat, points="all")
    fig.update_yaxes( 
        title_text = "Price Per SF",
        title_standoff = 10)
    fig.update_xaxes(
        title_text = cat,
        title_standoff = 10)
    fig.show()
    return 

def indicatorConversion(df, cat):
    df[cat]=np.where(df[cat]>0, 1, 0)
    df[cat]=df[cat].astype('object')
    return df

In [7]:
df.drop_duplicates(keep='first', inplace=True)

In [8]:
ord_col = [ 'LotShape', 'LandContour',  'LandSlope' ,'ExterQual', 'ExterCond',  'BsmtQual', 'BsmtCond', 'BsmtExposure',
           'BsmtFinType1', 'BsmtFinType2',  'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu',
            'GarageFinish', 'GarageQual','GarageCond', 'PavedDrive', 'PoolQC', 'Fence' ]
cat_col = ['MSSubClass', 'MSZoning', 'Neighborhood','Street', 'Alley', 'Utilities', 'LotConfig','Condition1', 'Condition2',  
          'BldgType', 'HouseStyle','RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd','MasVnrType','Foundation', 'Heating',
          'GarageType','MiscFeature','SaleType', 'SaleCondition'
         ]
cat_names = ['LotShape', 'LandContour',  'LandSlope' ,'ExterQual', 'ExterCond',  'BsmtQual', 'BsmtCond', 'BsmtExposure',
            'BsmtFinType1', 'BsmtFinType2',  'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu',
             'GarageFinish', 'GarageQual','GarageCond', 'PavedDrive', 'PoolQC', 'Fence','MSSubClass','MSZoning', 'Neighborhood','Street', 'Alley', 'Utilities', 'LotConfig','Condition1', 'Condition2',  
           'BldgType', 'HouseStyle','RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd','MasVnrType','Foundation', 'Heating',
           'GarageType','MiscFeature','SaleType', 'SaleCondition']

df[ord_col]=df[ord_col].astype("object")
df[cat_col]=df[cat_col].astype("category")

In [9]:
X=df.drop(['SalePrice'], axis = 1)
y=df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.3, random_state=42)

In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 135 to 1126
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Id             1022 non-null   int64   
 1   MSSubClass     1022 non-null   category
 2   MSZoning       1022 non-null   category
 3   LotFrontage    832 non-null    float64 
 4   LotArea        1022 non-null   int64   
 5   Street         1022 non-null   category
 6   Alley          66 non-null     category
 7   LotShape       1022 non-null   object  
 8   LandContour    1022 non-null   object  
 9   Utilities      1022 non-null   category
 10  LotConfig      1022 non-null   category
 11  LandSlope      1022 non-null   object  
 12  Neighborhood   1022 non-null   category
 13  Condition1     1022 non-null   category
 14  Condition2     1022 non-null   category
 15  BldgType       1022 non-null   category
 16  HouseStyle     1022 non-null   category
 17  OverallQual    1022 non-null   

In [11]:
categorical_columns2=X_train.select_dtypes(include={ 'category'}).columns

In [12]:
preprocessor=make_column_transformer(
    (
        OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1),
        make_column_selector(dtype_include='object')
    ),
   (
        OneHotEncoder(handle_unknown='ignore'),
        categorical_columns2
    ),
    remainder="passthrough",
    verbose_feature_names_out=True,
        
)


In [13]:
hist_native = make_pipeline(
    preprocessor,
    HistGradientBoostingRegressor(
        random_state=42,
        categorical_features=range(preprocessor.fit(X_train).output_indices_['ordinalencoder'].stop)
    ),
)

In [14]:
cross_validate(hist_native, X_train, y_train, cv=10, scoring="neg_mean_absolute_percentage_error")

{'fit_time': array([3.91685486, 3.86491108, 3.79142952, 3.74796891, 3.84966493,
        3.80287504, 3.72250986, 3.76937866, 3.80601907, 3.92854857]),
 'score_time': array([0.00899935, 0.01200008, 0.00999951, 0.00999975, 0.0090003 ,
        0.00900745, 0.0091033 , 0.01007771, 0.00900054, 0.01199961]),
 'test_score': array([-0.09721808, -0.09261802, -0.12323027, -0.11755257, -0.11714984,
        -0.08775637, -0.10043247, -0.08527988, -0.0878749 , -0.07986109])}

cross_validate(hist_native, X2_train, y2_train, cv=5, scoring="neg_mean_absolute_percentage_error")

In [15]:
hist_native.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001ACC0360B50>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['MSSubClass', 'MSZoning...Config',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'Foundation', 'Heating', 'GarageType', 'MiscFeature', 'SaleType',
       'SaleCondition'],
      dtype='object'))])),
                ('histgradientboostingregressor',
                 HistGradientBoostingRegressor(categorical_features=range(0, 22),
                                               random_state=42))])

In [16]:
X_results=hist_native.predict(X_test)

np.log(X_results*X_test['GrLivArea'])

In [17]:
def logSaleprice(pricePerSF, df):
    return np.log(pricePerSF*df['GrLivArea'])
    

In [18]:
dftest= pd.read_csv("test.csv")

In [19]:
Idcol=dftest['Id']

In [20]:
test_results=pd.Series(hist_native.predict(dftest))

In [21]:
test_results

0       120814.446882
1       155631.117151
2       188021.605329
3       195566.163686
4       204225.091570
            ...      
1454     78574.948742
1455     88956.085676
1456    149107.661510
1457    118883.890369
1458    213859.549132
Length: 1459, dtype: float64

In [22]:
final_result=pd.concat((Idcol,test_results), axis = 1)

In [23]:
final_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      1459 non-null   int64  
 1   0       1459 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 22.9 KB


In [24]:
final_result.rename(columns={0:'SalePrice'},inplace= True)

In [25]:
final_result.to_csv('final_result_baseline.csv', index=False)